In [21]:
from common import *
from experiment import *
from data import DirectRawData, RawData, PlottableData

In [22]:
GROUPS = ['cubic', 'bbr1', 'bbr2', 'bbr3']
TREATMENTS = [
    TCPTreatment(cca='cubic', pep=False, label='tcp_cubic'),
    TCPTreatment(cca='bbr', pep=False, label='tcp_bbr'),
    TCPTreatment(cca='bbr2', pep=False, label='tcp_bbr2'),
    TCPTreatment(cca='bbr', pep=False, label='tcp_bbr1'),  # When the default bbr module on the current Linux kernel is v1!
    TCPTreatment(cca='cubic', pep=True, label='pep_cubic'),
    TCPTreatment(cca='bbr', pep=True, label='pep_bbr'),
    TCPTreatment(cca='bbr2', pep=True, label='pep_bbr2'),
    TCPTreatment(cca='bbr', pep=True, label='pep_bbr1'),  # When the default bbr module on the current Linux kernel is v1!
]
TCP_CUBIC = TREATMENTS[0]
TCP_BBRV3 = TREATMENTS[1]
TCP_BBRV2 = TREATMENTS[2]
TCP_BBRV1 = TREATMENTS[3]
PEP_CUBIC = TREATMENTS[4]
PEP_BBRV3 = TREATMENTS[5]
PEP_BBRV2 = TREATMENTS[6]
PEP_BBRV1 = TREATMENTS[7]

# Pick network settings

In [23]:
# Generate the treatments
treatments_map = {
    'pep': [PEP_CUBIC, PEP_BBRV1, PEP_BBRV2, PEP_BBRV3],
    'tcp': [TCP_CUBIC, TCP_BBRV1, TCP_BBRV2, TCP_BBRV3],
}

In [24]:
# Generate a representative network setting and data size
num_trials = 10
network_settings = [
    NetworkSetting(delay1=1, delay2=80, loss1='0', loss2='4', bw1=10, bw2=50),
    NetworkSetting(delay1=1, delay2=20, loss1='4', loss2='0', bw1=30, bw2=30),
    NetworkSetting(delay1=1, delay2=20, loss1='4', loss2='0', bw1=30, bw2=30).mirror(),
    NetworkSetting(delay1=20, delay2=20, loss1='2', loss2='2', bw1=50, bw2=50),
    NetworkSetting(delay1=1, delay2=20, loss1='3', loss2='1', bw1=30, bw2=30),
]
data_size = lambda bw: int(10*1000000*bw/8)  # 10x the bottleneck bandwidth
data_sizes = [data_size(min(ns.get('bw1'), ns.get('bw2'))) for ns in network_settings]

# Execute experiments for each qdisc

In [25]:
# TODO: Set the index for the appropriate kernel!
index = 1

In [26]:
def network_settings_with_qdisc(network_settings, qdisc):
    ns_qdiscs = []
    for ns in network_settings:
        ns_qdisc = ns.clone()
        ns_qdisc.set('qdisc', qdisc)
        ns_qdiscs.append(ns_qdisc)
    return ns_qdiscs

def network_setting_with_qdiscs(network_setting, qdiscs):
    ns_qdiscs = []
    for qdisc in qdiscs:
        ns_qdisc = network_setting.clone()
        ns_qdisc.set('qdisc', qdisc)
        ns_qdiscs.append(ns_qdisc)
    return ns_qdiscs

In [ ]:
qdiscs = [
    'red',
    'bfifo-large',
    'bfifo-small',
    'pie',
    'policer',
    # 'codel',
]
for qdisc in qdiscs:
    # Define the experiment
    exp = Experiment(
        num_trials=1,
        treatments=[
            treatments_map['pep'][index],
            treatments_map['tcp'][index],
        ],
        network_settings=network_settings_with_qdisc(network_settings, qdisc),
        data_sizes=data_sizes,
        cartesian=False,
    )

    # Parse data
    execute = True
    raw_data = RawData(exp, execute=execute)

5
13.68497896194458mulation/main.py --bw1 10 --bw2 50 --delay2 80 --loss1 0 --loss2 4 --qdisc red -t 1 --label pep_bbr1 tcp --pep -cca bbr -n 12500000 
11.643951416015625ulation/main.py --bw1 30 --bw2 30 --delay2 20 --loss1 4 --qdisc red -t 1 --label pep_bbr1 tcp --pep -cca bbr -n 37500000 
11.664195775985718ulation/main.py --bw1 30 --bw2 30 --delay1 20 --delay2 1 --loss1 0 --loss2 4 --qdisc red -t 1 --label pep_bbr1 tcp --pep -cca bbr -n 37500000 
14.396405220031738ulation/main.py --bw1 50 --bw2 50 --delay1 20 --delay2 20 --loss1 2 --loss2 2 --qdisc red -t 1 --label pep_bbr1 tcp --pep -cca bbr -n 62500000 
11.843924522399902ulation/main.py --bw1 30 --bw2 30 --delay2 20 --loss1 3 --loss2 1 --qdisc red -t 1 --label pep_bbr1 tcp --pep -cca bbr -n 37500000 
5
13.46788740158081mulation/main.py --bw1 10 --bw2 50 --delay2 80 --loss1 0 --loss2 4 --qdisc bfifo-large -t 1 --label pep_bbr1 tcp --pep -cca bbr -n 12500000 
12.743804931640625ulation/main.py --bw1 30 --bw2 30 --delay2 20 --loss1 4 -

# Plotting functions

In [ ]:
def compare_qdiscs(data, qdiscs, ns_qdiscs, data_size, group, network_setting, delta=25, ylabel='Goodput (Mbit/s)'):
    treatments = data.treatments

    # Bar width and positions
    bar_width = 1 / len(qdiscs) - 0.01
    x = np.arange(len(treatments))

    # Create the plot
    fig, ax = plt.subplots(figsize=(8, 6))

    # Add bars for each qdisc
    for i, (qdisc, network_setting) in enumerate(zip(qdiscs, ns_qdiscs)):
        ys = []
        yerr_lower = []
        yerr_upper = []

        for treatment in treatments:
            subdata = data.data[treatment][network_setting.label()].get(data_size)
            if subdata is None:
                ys.append(0)
                yerr_lower.append(0)
                yerr_upper.append(0)
                continue
            ys.append(subdata.p(50))
            yerr_lower.append(subdata.p(50) - subdata.p(50-delta))
            yerr_upper.append(subdata.p(50+delta) - subdata.p(50))

        ax.bar(x - bar_width/2 + i*bar_width, ys, bar_width, yerr=(yerr_lower, yerr_upper), label=qdisc)
        
    # Labeling and formatting
    ax.set_xlabel('Congestion Control Algorithm')
    ax.set_ylabel(ylabel)
    ax.set_title(f'{group} {network_setting.label()}')
    ax.set_xticks(x)
    ax.set_xticklabels(treatments)
    ax.grid()
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=len(qdiscs))
    
    # Show the plot
    plt.tight_layout()
    plt.show()

# Plot data

In [ ]:
for ns_index, (network_setting, data_size) in enumerate(zip(network_settings, data_sizes)):
    ns_qdiscs = network_setting_with_qdiscs(network_setting, qdiscs)
    for group, treatments in treatments_map.items():
        # Define the experiment
        exp = Experiment(
            num_trials=50,
            treatments=treatments,
            network_settings = ns_qdiscs,
            data_sizes = [data_size],
        )

        # Parse data
        raw_data = RawData(exp, execute=False)
        
        # Convert to plottable data
        metric = 'throughput_mbps'
        plottable_data = PlottableData(raw_data, metric=metric)
        compare_qdiscs(plottable_data, qdiscs, ns_qdiscs, data_size, f'{group} #{ns_index}', network_setting)